In [1]:
#https://stackoverflow.com/questions/12919980/nohup-is-not-writing-log-to-output-file
# nohup python -u ./gte_vae.py > gte_vae.log &
# nohup python -u ./gte_vae_pretrain.py > gte_vae_pretrain.log &
%load_ext autoreload
%autoreload 2
import os, sys
import time
import math
import glob

import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
from torchtext import data
from torchtext import datasets
from argparse import ArgumentParser

In [2]:
import re
from custom_snli_loader import CustomSNLI
from enc_dec import EncDec

def tokenize(sent):
    '''
    data_reader.tokenize('a#b')
    ['a', '#', 'b']
    '''
    sent = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", sent) #.
    #if 'is is '
    return [x.strip().lower() for x in re.split('(\W+)?', sent) if x.strip()]

max_seq_len = 52#35
vocab_size = 10000
#inputs = data.Field(lower=False, fix_length=max_seq_len, init_token='<sos>')
inputs = data.Field(lower=False, fix_length=max_seq_len, tokenize=tokenize, init_token='<sos>')
answers = data.Field(sequential=False)
train, dev, test = CustomSNLI.splits(inputs, answers)
print('custom data loading debug:')
print(len(train))
print(len(dev))
print(len(test))
inputs.build_vocab(train, dev, test, max_size=vocab_size)

vector_cache = os.path.join(os.getcwd(), '.vector_cache/input_vectors.pt')
word_vectors = 'glove.42B.300d'

if os.path.isfile(vector_cache):
    inputs.vocab.vectors = torch.load(vector_cache)
else:
    #inputs.vocab.load_vectors(wv_dir=args.data_cache, wv_type=args.word_vectors, wv_dim=args.d_embed)
    print('Loading word embeddings...')
    
    inputs.vocab.load_vectors(word_vectors)
    makedirs(os.path.dirname(vector_cache))
    torch.save(inputs.vocab.vectors, vector_cache)

/home/paperspace/.pyenv/versions/3.6.3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


custom data loading debug:
183416
3329
3368


In [15]:
gpu = 0
#resume = '3-gru_11192017/lstm_gte_best'#'lstm_gte_e22'
#resume = '3-gru-1e-4_11202017/lstm_gte_best'
#resume = 'lstm_gte_e45'
#resume = 'vnmt_finetune_3-gru/vnmt_gru_gte_e15.pkl'
#resume = 'vnmt_pretrain'
#resume = 'vnmt_gru_gte_best.pkl'

#resume = 'vnmt_finetune_12052017/vnmt_gru_gte_best.pkl'
#resume = '12102017_0/vnmt_gru_gte_best.pkl'

#resume = 'vnmt_finetune_1-gru_5e-4_reverseinit_12092017/vnmt_gru_gte_best.pkl'
#resume = 'vnmt_finetune_1-gru_5e-4_sameinit_12092017/vnmt_gru_gte_best.pkl'
#resume = 'vnmt_fromscratch_1-gru_1e-4_12082017/vnmt_gru_gte_best.pkl'
print('test')


#resume = 'vnmt_gru_gte_e4.pkl'
#resume = 'vnmt_gru_gte_e10.pkl' # val 20
#resume = 'vnmt_gru_gte_e15.pkl' # val 20.80
#resume = 'vnmt_gru_gte_best.pkl'
#resume = 'vnmt_finetune_sameinit_afterbugfix_12112017/vnmt_gru_gte_best.pkl'
#resume = 'vnmt_finetune_sameinit_afterbugfix_12112017/vnmt_gru_gte_e15.pkl'
#resume = 'vnmt_finetune_rev?_afterbugfix_12122017/vnmt_gru_gte_best.pkl'
#resume = 'vnmt_finetune_1-gru_afterbugfix_same_12182017/vnmt_gru_gte_best.pkl'

resume = 'vnmt_fts_12192017_maxlength52/vnmt_gru_gte_best.pkl'

if os.path.isfile(resume):
    print("=> loading checkpoint '{}'".format(resume))
    model = torch.load(resume, map_location=lambda storage, locatoin: storage.cuda(gpu))
    model.encoder_prior.embeddings.weight.data = inputs.vocab.vectors
    model.encoder_post.embeddings.weight.data = inputs.vocab.vectors
    model.decoder.embeddings.weight.data = inputs.vocab.vectors
    model.encoder_prior.cuda()
    model.encoder_post.cuda()
    model.decoder.cuda()

    #print("=> loaded checkpoint '{}' (epoch {})"
    #      .format(resume, checkpoint['epoch']))
else:
    print("=> no checkpoint found at '{}'".format(resume))

test
=> loading checkpoint 'vnmt_fts_12192017_maxlength52/vnmt_gru_gte_best.pkl'


In [16]:
from gte import create_example

In [17]:
ntokens = len(inputs.vocab)

sents = [
    'People are celebrating a victory on the square',
    'A wedding party looks happy on the picture',
    'Two women who just had lunch hugging and saying goodbye.',

    'People are running away from the bear',
]


import numpy as np
np.random.seed(1234)
print('test')
for sent in sents:
    print('#'*50)
    print('#'*50)
    print(len(sent.split()))
    print(sent)

    example = create_example(inputs, sent, max_seq_len)
    print(example)
    #print(generate(model, ntokens, example))
    print(model.generate(inputs, ntokens, example, max_seq_len))
    #generate2(model, ntokens, example)

test
##################################################
##################################################
8
People are celebrating a victory on the square
[0, 10, 1118, 3, 2348, 9, 5, 891, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['<sos>', 'the', 'people', 'are', 'outside', '.', '<pad>']
##################################################
##################################################
8
A wedding party looks happy on the picture
[0, 563, 456, 110, 485, 9, 5, 95, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['<sos>', 'a', 'person', 'is', 'getting', 'a', 'picture', '.', '<pad>']
##################################################
##################################################
10
Two women who just had lunch hugging and saying goodbye.
[0, 50, 182, 511, 1014, 1217, 762, 11, 1957, 0, 1, 1, 1, 1, 1, 1, 1, 1, 

In [7]:
#from gte_vae import evaluate
from utils import get_args, makedirs, tokenize, load_dataset

def evaluate(val_iter, model, n_tokens, eval_batch_size, kld_weight=1.0, wv=None):
    """
    Eval acc, bleu, etc.
    """

    # Turn on evaluation mode which disables dropout.
    model.eval()
    model.encoder_prior.eval()
    model.encoder_post.eval()
    model.decoder.eval()
    total_loss = 0
    loss = 0

    for batch_idx, batch in enumerate(val_iter):
        batch.premise.data = batch.premise.data.transpose(1,0)
        batch.hypothesis.data = batch.hypothesis.data.transpose(1,0)
        _loss, _kld = model.batchNLLLoss(batch.premise, batch.hypothesis, train=False)
        loss += _loss + kld_weight * _kld # full kld when evaluation?

    return loss / float(len(val_iter))
batch_size=250
inputs, train_iter, val_iter, test_iter = load_dataset(batch_size, max_seq_len, vocab_size, word_vectors, vector_cache)

/home/gota/.pyenv/versions/3.6.2/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


KeyboardInterrupt: 

In [54]:
val_loss = evaluate(val_iter, model, ntokens, batch_size, kld_weight=1.0)
val_loss = val_loss.data[0]
print(val_loss)

20.801570892333984


/home/gota/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


In [8]:
ntokens = len(inputs.vocab)

sents = [
    'People are celebrating a victory on the square',
    'A wedding party looks happy on the picture',
    'Two women who just had lunch hugging and saying goodbye.',
    #'The athletes run at the start of the Haspa Hamburg marathon',
    'People are running away from the bear',
    
    'A person on a horse jumps over a broken down airplane.',
    'A boy is jumping on skateboard in the middle of a red bridge.',
    'High fashion ladies wait outside a tram beside a crowd of people in the city.',
    #'Woman in white in foreground and a man slightly behind walking with a sign for John\'s Pizza and Gyro in the background.',
    #'A woman is walking across the street eating a banana, while a man is following with his briefcase.',
    'A big brown dog swims towards the camera.',
    'A small group of church-goers watch a choir practice.',
    
    'A man is surfing in a bodysuit in beautiful blue water.',
    
    'First time a computer Go program has beaten a 9-dan professional without handicaps',
    'Competitors must be dressed in suitable tennis attire that is almost entirely white.',
    'The athletes run by the harbour during the Haspa Hamburg marathon.',
    'Space travellers gaze up and strain to find the blue dot in their sky. ',
    
    'University is designing a computer science education program to fulfill one of government \'s new education goals for the state.',
    'an airplane with skis parked in the water alongside a dock.',
    'a desk with a computer monitor and air plane model.',
    'cat looking in the mirror while the camera flash makes his eyes light up.',
    'a smiling guy sitting at a table with some food.',
    'a plate with an upside down cake on it and a bowl of fruit next to the plate.',
    'a young woman walking down the street while talking on her cell phone',
]

#model.eval()
import numpy as np
np.random.seed(1234)
print('test')
for sent in sents:
    print('#'*50)
    print('#'*50)
    print(len(sent.split()))
    print(sent)

    example = create_example(inputs, sent, max_seq_len)
    print(example)
    #print(generate(model, ntokens, example))
    print(model.generate(inputs, ntokens, example, max_seq_len))
    #generate2(model, ntokens, example)
    

test
##################################################
##################################################
8
People are celebrating a victory on the square
[0, 10, 1118, 3, 2348, 9, 5, 891, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['<sos>', 'the', 'people', 'are', 'outside', '.', '<pad>']
##################################################
##################################################
8
A wedding party looks happy on the picture
[0, 563, 456, 110, 485, 9, 5, 95, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['<sos>', 'a', 'person', 'is', 'getting', 'a', 'picture', '.', '<pad>']
##################################################
##################################################
10
Two women who just had lunch hugging and saying goodbye.
[0, 50, 182, 511, 1014, 1217, 762, 11, 1957, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['<sos>', 'a', 'woman', 'is', 'hugging', 'a', '<pad

In [18]:
import nltk

def check_val(test_set=dev):
    max_seq_len=60
    max_length=35
    longest = 0
    max_count=0
    skipped = 0
    interval=5.0
    length_count = [0] * int(max_seq_len/interval)
    
    for pair_idx, pair in enumerate(test_set):
        input = pair.premise
        #print(input) => input without sos/eos symbols
        
        length = len(input)
        if len(input) >= max_length-1:
            skipped += 1
        if length>max_length:
            max_count+=1
            #print(length)
        if longest<length:
            longest = length
            #print(longest)
            #print(max_count)
            
        for i in range(len(length_count)):
            if length > i*interval and length <= (i+1)*interval:
                length_count[i] += 1
    print('skipeed=%d'%skipped)
    print(longest)
    print(max_count)
    print(length_count)


#def evaluate(val_iter, model, n_tokens, eval_batch_size, wv=None):
def evaluate_bleu(filename, test_set=dev):
    """
    Eval acc, bleu, etc.
    """

    # Turn on evaluation mode which disables dropout.
    model.encoder_prior.eval()
    model.decoder.eval()
    mean_bleu = 0
    mean_bleu_long=0
    count = 0
    skipped = 0
    
    # take mean by lengths
    interval = 5
    max_seq_len=52
    mean_bleu_length = [0] * int((max_seq_len-2)/interval)
    mean_bleu_count = [0] * int((max_seq_len-2)/interval)
    
    max_length=52
    longest = 0
    max_count=0
    
    for pair_idx, pair in enumerate(test_set):
        #input = ['<sos>']+pair.premise+['<pad>']
        input = pair.premise
        
        if len(input) >= max_seq_len-1:
            skipped += 1
            continue
        
        example = create_example(inputs, ' '.join(input), len(input))
        prediction = model.generate(inputs, ntokens, example, max_seq_len)
        #bleu = nltk.translate.bleu_score.sentence_bleu(input, prediction)
        #print(pair.hypothesis)
        
        # IMPORTANT: eliminate <sos> and <pad>
        prediction = prediction[1:len(prediction)-1]
        
        bleu = nltk.translate.bleu_score.sentence_bleu(pair.hypothesis, prediction)
        mean_bleu += bleu
        
        '''
        print(input)
        print(example)
        print(pair.hypothesis)
        print(prediction)
        print(prediction[1:len(prediction)-1])
        print(bleu)'''
        
        # export to file for later evaluation
        #with open('results/dev_reference.txt', 'a') as the_file:
        #    the_file.write(' '.join(pair.hypothesis) + '\n')
        
        with open('results/%s.txt'%filename, 'a') as the_file:
            the_file.write(' '.join(prediction) + '\n')
            
        length = len(input)
        for i in range(len(mean_bleu_length)):
            if length > i*interval and length <= (i+1)*interval:
                with open('results/%s_len%d.txt'%( filename, (i+1)*interval ), 'a') as the_file:
                    the_file.write(' '.join(prediction) + '\n')
                # save refernce as well
                #with open('results/dev_reference_len%d.txt'%( (i+1)*interval ), 'a') as the_file:
                #    the_file.write(' '.join(pair.hypothesis) + '\n')
                mean_bleu_length[i] += bleu
                mean_bleu_count[i] += 1
        
            
        
        if len(pair.premise) > 20:
            mean_bleu_long += bleu
            count+=1
            
        
        if pair_idx % 100 == 0 and pair_idx > 0:
            print(pair_idx)
            print(mean_bleu/float(pair_idx-skipped))
            #fds
        
    for i in range(len(mean_bleu_length)):
        if mean_bleu_count[i] != 0:
            mean_bleu_length[i] /= float(mean_bleu_count[i])
    print('skipped examples=%d'%skipped)
    print('mean bleus by length:')
    print(mean_bleu_length)
    print('sent counts by length:')
    print(mean_bleu_count)
    print('mean bleu for longer sentences')
    print( mean_bleu_long/float(count) )
    
    return mean_bleu/float(len(test_set)-skipped)


In [32]:
check_val(dev)
print('='*50)
#check_val(dev)

skipeed=68
59
46
[29, 814, 1123, 668, 448, 152, 49, 28, 5, 7, 4, 2]


In [19]:
#filename = 'dev_vnmt_finetune_rev'
filename = 'dev_vnmt_finetune'
print(evaluate_bleu(filename, dev))
#80
#0.5433655936017571
#0.4969424975986079

/home/gota/.pyenv/versions/3.6.2/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


100
0.6746799172878135
200
0.6852496095247629
300
0.6873491870712055
400
0.6764574079743771
500
0.6700578917874082
600
0.6718738118937818
700
0.6757725546737736
800
0.6712232363624427
900
0.6716525241184327
1000
0.672557117752946
1100
0.6724721536409493
1200
0.6741256857828162
1300
0.6708358655122696
1400
0.6705935862798202
1500
0.6708392713420225
1600
0.6707662623229714
1700
0.6715312380054739
1800
0.6724492105381966
1900
0.6735502114943078
2000
0.6734427304743983
2100
0.6728480454262491
2200
0.6738363875162604
2300
0.6735308427318685
2400
0.6729581955363363
2500
0.673248483048291
2600
0.6744996307040293
2700
0.6751602368648107
2800
0.6756106533014942
2900
0.6764793374571778
3000
0.6776203288404397
3100
0.6773346474422215
3200
0.6772059389982139
3300
0.6775760275874917
skipped examples=6
mean bleus by length:
[0.6219907774353548, 0.6680750317683846, 0.6818619980748704, 0.680946327231944, 0.6761950838085073, 0.6779750919500189, 0.6755346439664616, 0.7196724321454913, 0.6991519714705647

In [ ]:
print('=============')
print(evaluate_bleu(test))

In [ ]:
### Results:
# on test set, 1-gru/1-gru model: 
#69
# pad
#1206: 0.47339631549088546
#1208: 0.478161921044296
# w/o pad:
# 0.676472029311804
#0.6719636397772298  <=> 0.6719217893158086

# finetune
#0.671403541792545
#0.6682873087200726
#


# dev
#including pad
#0.48138155256013737
#0.4680436671127621
# w/o pad
#0.675083768523707 <=> 0.6777948048325388
#0.662513162802644 <=> 0.6618760696325411

#finetune
#0.6722088426694864
#0.6592629383357872


# on test set, 3-gru/1-gru model:

In [ ]:
0.0
0.7215811598543795 # 3-GRU w/o attention when trained w/ 1e-3
train_iter, val_iter, test_iter = data.BucketIterator.splits(
            (train, dev, test), batch_size=256, device=0)
len(train_iter)

In [26]:
def evaluate_val_pred():
    # Turn on evaluation mode which disables dropout.
    model.encoder_prior.eval()
    model.encoder_post.eval()
    model.decoder.eval()
    mean_bleu = 0
    print(max_seq_len)
    for pair_idx, pair in enumerate(dev):
        
        
        #input = pair.premise
        input = ['<sos>']+pair.premise+['<pad>']
        #print(input)
        if len(input) > max_seq_len-1:
            continue
        
        #print(type(input))
        #print(' '.join(input))
        if len(input) > 20:
            example = create_example(inputs, ' '.join(input), len(input))
            #print(example)
            
            prediction = model.generate(inputs, ntokens, example, max_seq_len)
            print('#'*50)
            print(input)
            print(pair.hypothesis)
            print(prediction)

        if pair_idx % 500 == 0 and pair_idx > 0:
            return

evaluate_val_pred()

35
##################################################
['<sos>', 'a', 'man', 'in', 'a', 'blue', 'shirt', 'standing', 'in', 'front', 'of', 'a', 'garage', '-', 'like', 'structure', 'painted', 'with', 'geometric', 'designs', '.', '<pad>']
['a', 'man', 'is', 'wearing', 'a', 'blue', 'shirt']
['<sos>', 'a', 'man', 'is', 'standing', 'in', 'front', 'of', 'a', 'house', '.', '<pad>']
##################################################
['<sos>', 'under', 'a', 'blue', 'sky', 'with', 'white', 'clouds', ',', 'a', 'child', 'reaches', 'up', 'to', 'touch', 'the', 'propeller', 'of', 'a', 'plane', 'standing', 'parked', 'on', 'a', 'field', 'of', 'grass', '.', '<pad>']
['a', 'child', 'is', 'reaching', 'to', 'touch', 'the', 'propeller', 'of', 'a', 'plane', '.']
['<sos>', 'a', 'child', 'is', 'outside', '.', '<pad>']
##################################################
['<sos>', 'man', 'in', 'a', 'black', 'suit', ',', 'white', 'shirt', 'and', 'black', 'bowtie', 'playing', 'an', 'instrument', 'with', 'the', 'rest'

##################################################
['<sos>', 'a', 'man', 'in', 'a', 'black', 'shirt', 'is', 'smiling', 'at', 'a', 'woman', 'in', 'a', 'black', 'shirt', 'what', 'a', 'tattoo', 'and', 'a', 'eye', 'brow', 'ring', '.', '<pad>']
['a', 'man', 'is', 'smiling', 'at', 'a', 'woman', '.']
['<sos>', 'a', 'man', 'is', 'smiling', '.', '<pad>']
##################################################
['<sos>', 'a', 'goalie', 'in', 'white', 'runs', 'for', 'an', 'approaching', 'ball', 'while', 'the', 'opponent', 'in', 'red', 'who', 'kicked', 'it', 'waits', '.', '<pad>']
['a', 'person', 'wearing', 'white', 'is', 'running', 'towards', 'a', 'ball', 'that', 'was', 'kicked', 'from', 'another', 'person', '.']
['<sos>', 'a', 'goalie', 'is', 'playing', 'a', 'game', '.', '<pad>']
##################################################
['<sos>', 'two', 'men', 'sitting', 'on', 'a', 'subway', 'are', 'reading', ',', 'with', 'coats', 'and', 'scarves', 'on', ',', 'but', 'have', 'seemed', 'to', 'have', 'lost', 't

##################################################
['<sos>', 'two', 'girls', 'with', 'long', 'brown', 'hair', 'sit', 'on', 'the', 'floor', 'with', 'a', 'view', 'from', 'the', 'window', 'behind', 'them', 'showing', 'a', 'town', '.', '<pad>']
['two', 'girls', 'with', 'long', 'brown', 'hair', 'sit', 'on', 'the', 'floor']
['<sos>', 'two', 'girls', 'are', 'sitting', 'on', 'a', 'stoop', '.', '<pad>']
##################################################
['<sos>', 'man', 'in', 'a', 'military', 'outfit', 'handing', 'something', 'in', 'a', 'plastic', 'bag', 'to', 'a', 'group', 'of', 'children', 'outside', 'near', 'a', 'brown', 'building', '.', '<pad>']
['a', 'man', 'is', 'handing', 'a', 'plastic', 'bag', 'to', 'a', 'group', 'of', 'children', '.']
['<sos>', 'a', 'man', 'is', 'in', 'front', 'of', 'a', 'group', 'of', 'children', '.', '<pad>']
##################################################
['<sos>', 'a', 'man', 'and', 'a', 'woman', 'sit', 'on', 'a', 'deck', 'under', 'a', 'blue', 'sky', 'that', 'is

##################################################
['<sos>', 'the', 'woman', 'is', 'holding', 'up', 'the', 'line', 'with', 'the', 'fish', 'that', 'she', 'caught', 'in', 'the', 'lake', 'with', 'her', 'bamboo', 'fishing', 'pole', '.', '<pad>']
['the', 'woman', 'is', 'fishing', '.']
['<sos>', 'a', 'woman', 'is', 'fishing', '.', '<pad>']
##################################################
['<sos>', 'a', 'woman', 'wearing', 'gray', 'pants', ',', 'a', 'white', 'blouse', 'and', 'a', 'black', 'vest', 'is', 'jumping', 'with', 'one', 'hand', 'in', 'the', 'air', 'as', 'she', 'goes', 'through', 'an', 'indoor', 'stadium', '.', '<pad>']
['the', 'jumping', 'lady', 'in', 'slacks', 'also', 'has', 'her', 'hand', 'raised', '.']
['<sos>', 'a', 'woman', 'is', 'jumping', '.', '<pad>']
##################################################
['<sos>', 'a', 'brunette', 'woman', 'in', 'a', 'multicolored', 'floral', 'outfit', 'and', 'a', 'pearl', 'necklace', 'is', 'talking', 'while', 'holding', 'a', 'wineglass', '.', 

##################################################
['<sos>', 'a', 'woman', 'is', 'holding', 'one', 'young', 'children', 'with', 'another', 'standing', 'next', 'to', 'her', 'while', 'a', 'man', 'is', 'sitting', 'by', 'a', 'small', 'campfire', '.', '<pad>']
['a', 'woman', 'is', 'holding', 'one', 'young', 'children', 'with', 'another', 'standing', 'next', 'to', 'her']
['<sos>', 'a', 'woman', 'is', 'holding', 'a', 'baby', '.', '<pad>']
##################################################
['<sos>', 'a', 'basset', 'hound', 'is', 'tied', 'to', 'a', 'doorway', 'in', 'an', 'alley', 'in', 'front', 'of', 'a', 'man', 'and', 'woman', '.', '<pad>']
['the', 'dog', 'is', 'outside', '.']
['<sos>', 'a', 'man', 'is', 'in', 'front', 'of', 'a', 'garage', '.', '<pad>']
##################################################
['<sos>', 'a', 'man', 'in', 'a', 'plaid', 'shirt', 'and', 'brown', 'pants', 'walking', 'around', 'town', 'with', 'one', 'hand', 'in', 'his', 'pocket', '.', '<pad>']
['the', 'man', 'is', 'going'

##################################################
['<sos>', 'a', 'woman', 'with', 'blond', '-', 'hair', 'is', 'sitting', 'at', 'a', 'table', 'and', 'talking', 'to', 'a', 'man', 'with', 'glasses', '.', '<pad>']
['two', 'people', 'converse', 'at', 'a', 'table', '.']
['<sos>', 'a', 'woman', 'is', 'talking', 'to', 'a', 'man', '.', '<pad>']
##################################################
['<sos>', 'a', 'well', 'safety', 'equipment', 'biker', 'racing', 'on', 'the', 'trail', 'in', 'a', 'wooded', 'area', 'with', 'number', '569', 'on', 'his', 'bike', '<pad>']
['a', 'biker', 'races', 'on', 'the', 'trail']
['<sos>', 'a', 'biker', 'is', 'riding', 'a', 'bike', '.', '<pad>']
##################################################
['<sos>', 'an', 'african', '-', 'american', 'football', 'player', ',', 'wearing', 'jersey', 'number', '12', 'for', 'the', 'university', 'of', 'oklahoma', ',', 'throws', 'a', 'football', 'to', 'his', 'left', '.', '<pad>']
['a', 'man', 'throws', 'a', 'football', '.']
['<sos>',

In [ ]:
# check if z is continuous
